In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("loop.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])

In [77]:
def filter_by_normal(pcd):
    ind = []
    for i in range(len(pcd.points)):
        if ((np.abs(pcd.normals[i][2]) < 0.3) and ((np.abs(pcd.normals[i][0]) > 0.8) or (np.abs(pcd.normals[i][1]) > 0.8))):
            pcd.points[i][2] = np.random.rand()*0.01
            pass
        else:
            ind.append(i)
    result = pcd.select_by_index(ind, invert=True)
    return result

In [212]:
def compute_transform(vector1, vector2):
    # Normalize the vectors to unit length
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)

    # Compute the rotation matrix
    rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()

    # Compute the translation vector
    translation_vector = vector2 - np.dot(rotation_matrix, vector1)

    return translation_vector, rotation_matrix

In [213]:
def create_arrow_from_vector(vector, location):
    mesh_normal = vector
    mesh_center = location
    
    #print("Norm average:")
    #print(norm_average)
    
    arrow = o3d.geometry.TriangleMesh.create_cone(0.5,1)
    
    arrow.paint_uniform_color([1,0,0])
    arrow_bb = arrow.get_oriented_bounding_box()
    arrow.compute_triangle_normals()
    arrow.compute_vertex_normals()
    arrow_normal = np.asarray(arrow.vertex_normals)[0]

    translation, rotation = compute_transform(mesh_normal, arrow_normal)  
    
    arrow.rotate(rotation)
    
    arrow.translate(mesh_center, False)
    
    
   
    return arrow
    
    

In [216]:
arrow = create_arrow_from_vector([1,0,0], [-6.5,0,0])
o3d.visualization.draw_geometries([pcd_flat.paint_uniform_color([0,0,0])])

/tmp/ipykernel_6018/927153186.py:7: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()


In [151]:
def create_voxelgird_from_list(point_coordinates, voxel_size):
    pc = o3d.geometry.PointCloud()
    
    pc.points.extend(o3d.utility.Vector3dVector(point_coordinates))
    
    vg = o3d.geometry.VoxelGrid.create_from_point_cloud(pc, voxel_size)
    
    return vg
    

In [171]:
def is_array_in_list(target_array, array_list):
    for array in array_list:
        if np.array_equal(target_array, array):
            return True
    return False

In [180]:
def find_neighbour_voxels_2d(coordinate, voxel_size):
    
    # Define the possible offsets for the surrounding voxels in x, y, and z directions
    offsets = [(dx, dy)
        for dx in range(-1, 2)
        for dy in range(-1, 2)
        if (dx, dy) != (0, 0)
    ]

    # Calculate the surrounding voxel coordinates based on the offsets and voxel size
    surrounding_voxels = [
        np.asarray([
            coordinate[0] + dx * voxel_size,
            coordinate[1] + dy * voxel_size,
            coordinate[2],
        ])
        for dx, dy in offsets
    ]
    
    return surrounding_voxels


neighbours = find_neighbour_voxels_2d([-6.5,0,0], 0.1) 


for i in range(5):
    tmp = []
    
    for n in neighbours:
        new = find_neighbour_voxels_2d(n, 0.1)
        
        for x in new:
            if not (is_array_in_list(x, neighbours)):
                tmp.append(x)
    neighbours.extend(tmp)
    print(len(neighbours))
    test_vg =  create_voxelgird_from_list(neighbours, 0.1)  
    o3d.visualization.draw_geometries([test_vg, pcd_flat])

48
152
630
2080
6446


KeyboardInterrupt: 

In [203]:
def find_neighbour_voxels_2d2(process_data):
    coordinate, voxel_size = process_data
    # Define the possible offsets for the surrounding voxels in x, y, and z directions
    offsets = [(dx, dy)
        for dx in range(-1, 2)
        for dy in range(-1, 2)
        if (dx, dy) != (0, 0)
    ]

    # Calculate the surrounding voxel coordinates based on the offsets and voxel size
    surrounding_voxels = [
        np.asarray([
            coordinate[0] + dx * voxel_size,
            coordinate[1] + dy * voxel_size,
            coordinate[2],
        ])
        for dx, dy in offsets
    ]
    
    return surrounding_voxels


neighbours = find_neighbour_voxels_2d2(([-6.5,0,0], 0.1)) 
test_vg =  create_voxelgird_from_list(neighbours, 0.1)  
o3d.visualization.draw_geometries([test_vg, pcd_flat])

for i in range(4):
    tmp = []
    
    voxel_sizes = [0.1]*len(neighbours)
    process_data = zip(neighbours, voxel_sizes)
    with Pool(mp.cpu_count()) as p:
        new = p.map(find_neighbour_voxels_2d2, process_data)
    
    
    neighbours.extend(list(deepflatten(new, depth=1)))
    
    test_vg =  create_voxelgird_from_list(neighbours, 0.1)  
    o3d.visualization.draw_geometries([test_vg, pcd_flat])




Process ForkPoolWorker-1168:
Process ForkPoolWorker-1161:
Process ForkPoolWorker-1154:
Process ForkPoolWorker-1156:
Process ForkPoolWorker-1166:
Process ForkPoolWorker-1163:
Process ForkPoolWorker-1167:
Process ForkPoolWorker-1157:
Process ForkPoolWorker-1158:
Process ForkPoolWorker-1160:
Process ForkPoolWorker-1155:
Process ForkPoolWorker-1153:
Process ForkPoolWorker-1172:
Process ForkPoolWorker-1173:
Process ForkPoolWorker-1170:
Process ForkPoolWorker-1159:
Process ForkPoolWorker-1174:
Process ForkPoolWorker-1176:
Process ForkPoolWorker-1162:
Process ForkPoolWorker-1175:
Process ForkPoolWorker-1171:
Process ForkPoolWorker-1165:
Process ForkPoolWorker-1169:
Traceback (most recent call last):
Process ForkPoolWorker-1164:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent c

  File "/usr/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock

KeyboardInterrupt: 

In [149]:
def find_neighbour_voxels(vg, coordinate, voxel_size):
    
    # Define the possible offsets for the surrounding voxels in x, y, and z directions
    offsets = [(dx, dy, dz)
        for dx in range(-1, 2)
        for dy in range(-1, 2)
        for dz in range(-1, 2)
        if (dx, dy, dz) != (0, 0, 0)
    ]

    # Calculate the surrounding voxel coordinates based on the offsets and voxel size
    surrounding_voxels = [
        np.asarray([
            coordinate[0] + dx * voxel_size,
            coordinate[1] + dy * voxel_size,
            coordinate[2] + dz * voxel_size,
        ])
        for dx, dy, dz in offsets
    ]
    
    return surrounding_voxels


neighbours = find_neighbour_voxels(pcd_voxel, [-6.5,0,1], 0.1) 
neighbours2 = []
for n in neighbours:
    tmp = find_neighbour_voxels(pcd_voxel, n, 0.1)
    neighbours2.extend(tmp)
    


test_vg =  create_voxelgird_from_list(neighbours2, 0.1)  
o3d.visualization.draw_geometries([test_vg, pcd])

NameError: name 'pcd_voxel' is not defined

In [219]:
pcd_flat = filter_by_normal(pcd)

bb = pcd_flat.get_axis_aligned_bounding_box()
bb.color = np.asarray([0,0,1])
#mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(bb)

#pcd_sampled = mesh.sample_points_poisson_disk(5000)
o3d.visualization.draw_geometries([pcd_flat, mesh])

In [223]:
def create_uniform_pc_from_bb(bb_axis, voxel_size, color):
    pc = o3d.geometry.PointCloud()
    bb = o3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(bb_axis)
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(bb)
    mesh = mesh.paint_uniform_color([0,1,0])
    vg = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size)
    voxels = vg.get_voxels()
    grid_indexes = [x.grid_index for x in voxels]
    
    voxel_centers = [vg.get_voxel_center_coordinate(index) for index in grid_indexes]
    
    pc.points.extend(o3d.utility.Vector3dVector(voxel_centers))  
    pc.paint_uniform_color(color)
    return pc

uniform_pc = create_uniform_pc_from_bb(bb, 0.1, [1,0,0])
o3d.visualization.draw_geometries([pcd_flat, uniform_pc, bb])

In [224]:
def find_closest_vector_index(input_vector, pc):
    # Convert the input vector and vector list to numpy arrays for easier calculations
    input_vector = np.array(input_vector)
    vector_list = np.asarray(pc.points)

    # Calculate the Euclidean distances between the input vector and all vectors in the list
    distances = np.linalg.norm(vector_list - input_vector, axis=1)

    # Find the index of the vector with the minimum distance
    closest_index = np.argmin(distances)

    return closest_index

In [230]:
pcd_tree = o3d.geometry.KDTreeFlann(uniform_pc)
uniform_pc = uniform_pc.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([pcd_flat, uniform_pc])




print("Find its neighbors with distance less than 0.2, and paint them green.")

search_radius = np.ceil(np.sqrt(0.1**2+0.1**2)*10000)/10000

index = find_closest_vector_index([-6.5,0,1], uniform_pc)


[k, idx, _] = pcd_tree.search_radius_vector_3d(uniform_pc.points[index], search_radius)
np.asarray(uniform_pc.colors)[idx[1:], :] = [0, 1, 0]

new_neighbours = list(idx[1:])
old_set = set([])
new_set = set(new_neighbours)

tmp = []
for x in range(50):
    
    color = np.random.rand(3)
    
    
    different_elements = old_set.symmetric_difference(new_set)
    new_neighbours = list(different_elements)
    old_set = new_set
    
    for index in new_neighbours:
        [k, idx, _] = pcd_tree.search_radius_vector_3d(uniform_pc.points[index], search_radius)
        np.asarray(uniform_pc.colors)[idx[1:], :] = color
        tmp.extend(idx[1:])
    
    new_set = set(tmp)
    
    #test_pc = test_pc.select_by_index(tmp2, invert=True)
    

    o3d.visualization.draw_geometries([uniform_pc, pcd_flat])




Find its neighbors with distance less than 0.2, and paint them green.


In [152]:
pcd_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd.paint_uniform_color([1,0,0]), 0.1)

vg = create_voxelgird_from_list([[-6.5,0,1], [-6.6,0,1], [-6.4,0,1]], 0.1)
voxel_index = vg.get_voxel([-6,0,1])

voxels = pcd_voxel.get_voxels()
print(voxels[0])
#o3d.visualization.draw_geometries([pcd_voxel])

Voxel with grid_index: (169, 19, 10), color: (1, 0, 0)


In [ ]:
initial_seed = np.asarray([-6.5,0,0])
neighbours = find_neighbour_voxels_2d(initial_seed, 0.1)
test_vg =  create_voxelgird_from_list(neighbours, 0.1)  

o3d.visualization.draw_geometries([test_vg, pcd_flat])


    